In [9]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
import sklearn
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
import os 
import pathlib 
import random
import matplotlib.image as mpimg
import zipfile
import wget
import tensorflow_hub as hub

### Get Helper Functions

In [14]:
import requests
url = "https://raw.githubusercontent.com/Meta-c/Helper-ML/master/ML_functions.py"


response = requests.get(url)
code = response.text

exec(code)

### Getting Data

In [15]:
train_dir="10_food_classes_10_percent/train/"
test_dir="10_food_classes_10_percent/test/"

In [16]:
IMG_SIZE=(224,224)
BATCH_SIZE=32
train_data_10_percent= tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                            image_size=IMG_SIZE,label_mode="categorical",batch_size=BATCH_SIZE)

test_data=tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
image_size=IMG_SIZE,
label_mode="categorical",
batch_size=BATCH_SIZE)


Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [17]:
for images,labels in train_data_10_percent.take(1):
    print(images,labels)

tf.Tensor(
[[[[1.94849960e+02 1.57421402e+02 1.57778534e+02]
   [2.23080673e+02 1.86565201e+02 1.86565048e+02]
   [2.49104752e+02 2.16958221e+02 2.16672516e+02]
   ...
   [1.04346039e+02 8.76482162e+01 8.89053802e+01]
   [9.18103638e+01 8.12699203e+01 8.00959320e+01]
   [8.94947433e+01 7.97329483e+01 7.81375961e+01]]

  [[2.36570969e+02 2.16171570e+02 2.14655457e+02]
   [2.38884094e+02 2.18725143e+02 2.16487732e+02]
   [2.48470032e+02 2.28758789e+02 2.25178253e+02]
   ...
   [9.36241531e+01 7.98549042e+01 7.99842148e+01]
   [8.70481110e+01 7.93727875e+01 7.97539825e+01]
   [8.59735336e+01 7.98121796e+01 7.98306732e+01]]

  [[2.52838333e+02 2.51319992e+02 2.46280304e+02]
   [2.52389999e+02 2.52755096e+02 2.47659927e+02]
   [2.52107468e+02 2.50352524e+02 2.45352524e+02]
   ...
   [8.61834946e+01 7.25518417e+01 7.16991806e+01]
   [8.36509781e+01 7.66816635e+01 7.75864792e+01]
   [8.48437424e+01 8.04151688e+01 8.12008820e+01]]

  ...

  [[1.18174568e+02 1.23174568e+02 1.00174568e+02]
   [1